[View in Colaboratory](https://colab.research.google.com/github/todnewman/coe_training/blob/master/Complex_Data_Engineering_Example.ipynb)

# Complex Data Handling Example

## Overview

In this example, we do a few useful things that will be valuable as examples for how to clean and join data sources.  The data in this example comes from pro-football-reference.com, a site that is commonly scraped for fantasy football data.

The primary tool used here is Pandas, which is the python technique for building and manipulating dataframes.  Specifically, Pandas has a from_html method that can directly parse web tables.  This can be very useful for a number of data scraping activities because web tables can be a bit tricky.

## Data Description

For this example, we are blending two different types of tables from the site.  

*   The first table captures data for all players who had at least one rushing carry.  This means there are multiple positions represented in this table.
*   The second table captures data for all players who had receptions.  This means there is some overlap between this table and the rushing table.  

## Challenges

1.   Dealing with the overlap beween the two tables without duplication and without adding new features is important for developing good features that can be used for ML.  We redefine the headers ourselves to ensure we have good control of this.
2.   Doing the math and advanced lookups to build the labels (High Value record vs. Low Value record) from the existing data records is tricky and can be extremely time consuming if done in Excel offline.  Here I show examples of how to automate this.
3.   Cleanup of the data, filling holes, etc. is also repeatedly demonstrated.





## Install lxml

This step is needed to use the Pandas from_html method.  

In [16]:
!pip install lxml

In [0]:
import lxml
import html5lib
import pandas as pd

#
# Below we build a list that we'll use to pull multiple years of data from the web.
# I do this because its easier than typing up a list of all these years.
#

yr_list = []
for year in range(1992,2018):
  yr_list.append(year)


## Scrape Rushing Data Tables from the Web.

Grab data from the Rushing Leaders table for each year in our list.  Then we process and clean the data.  Calculating the score is best done here since the parameters differ across the types of tables on this site.  I also take this chance to strip off the special characters.  

This rushing data is interesing because it contains both rushing and receiving metrics.

In [0]:
'''
Grab data from the Rushing Leaders table for each year in our list.  Then we process and clean the data.  Calculating the score is best done here since the parameters
differ across the types of tables on this site.  I also take this chance to strip off the special characters.  This rushing data is interesing because it contains both
rushing and receiving metrics.  
'''

header = ['Rk', 'NAME', 'Team', 'Age', 'Pos', 'G', 'GS', 'Att', 'Rush Yds', 'Rushing TD', 'Rush Long', 'Y/A', 'Rush Y/G','A/G', 'Tgt', 'Rec', 'Rec Yds', 'Y/R', 'Rec TD', 'Rec Long',
          'R/G', 'Y/G', 'Catch%', 'Touch', 'APYd', 'Y/Tch', 'Yscm', 'RRTD', 'Fmb']
df_rush = pd.DataFrame()

for year in yr_list:
    filename = ("https://www.pro-football-reference.com/years/%s/rushing.htm" % year)

    tables = pd.read_html(filename, header=0)

    df_r = tables[0]
    df_r.columns = header
    df_r['Year'] = year
    df_rush = df_rush.append(df_r)    
filter = df_rush['Rk']=='Rk'
df_rush = df_rush[~filter]
df_rush = df_rush.fillna(0)
df_rush['Score'] = df_rush['Rush Yds'].astype(int)*.07 + df_rush['Rec Yds'].astype(int)*0.07 + df_rush['RRTD'].astype(int)*6 + df_rush['Rec'].astype(int)*0.5 - df_rush['Fmb'].astype(int)*2
#
# Remove strange characters from the right of the name using rstrip
#
df_rush['NAME'] = df_rush['NAME'].map(lambda x: x.rstrip('*'))
df_rush['NAME'] = df_rush['NAME'].map(lambda x: x.rstrip('+'))
df_rush['NAME'] = df_rush['NAME'].map(lambda x: x.rstrip('*'))

## Scrape Receiving Data Tables from the Web.

Grab data from the Receiving Leaders table for each year in our list.  Then we process and clean the data.  Calculating the score is best done here since the parameters differ across the types of tables on this site.  I also take this chance to strip off the special characters.  

This data only has receiving metrics.  There are some players in this data set who didn't rush a single yard and are therefore not captured in the rushing data already.

In [0]:
'''
Grab data from the Receiving Leaders table for each year in our list.  Then we process and clean the data.  Calculating the score is best done here since the parameters
differ across the types of tables on this site.  I also take this chance to strip off the special characters.  This data only has receiving metrics.  There are some
players in this data set who didn't rush a single yard and are therefore not captured in the rushing data already.
'''

header = ['Rk', 'NAME', 'Team', 'Age', 'Pos', 'G', 'GS', 'Tgt', 'Rec', 'Catch%', 'Rec Yds', 'Y/R', 'Rec TD', 'Rec Long',
          'R/G', 'Y/G', 'Fmb']
df_rec = pd.DataFrame()
for year in yr_list:
    filename = ("https://www.pro-football-reference.com/years/%s/receiving.htm" % year)

    tables = pd.read_html(filename, header=0)

    df_r = tables[0]
    df_r.columns = header
    df_r['Year'] = year
    df_rec = df_rec.append(df_r)
filter = df_rec['Rk']=='Rk'
df_rec = df_rec[~filter]
df_rec = df_rec.fillna(0)
df_rec['Score'] = df_rec['Rec Yds'].astype(int)*0.07 + df_rec['Rec TD'].astype(int)*6 + df_rec['Rec'].astype(int)*0.5 - df_rec['Fmb'].astype(int)*2
#
# Remove strange characters from the right of the name using rstrip
#
df_rec['NAME'] = df_rec['NAME'].map(lambda x: x.rstrip('*'))
df_rec['NAME'] = df_rec['NAME'].map(lambda x: x.rstrip('+'))
df_rec['NAME'] = df_rec['NAME'].map(lambda x: x.rstrip('*'))


## Concatenate Rushing and Receiving DataFrames

Here is where we concatenate the rushing dataframe with the records from the receiving dataframe that are NOT included in the rushing dataframe.

In [0]:
'''
Here is where we concatenate the rushing dataframe with the records from the receiving dataframe
that are NOT included in the rushing dataframe.
'''

target_value = 160  # This is the score above which we'll classify as a "high value" record

#
# Create a new key combined of name and year to determine how to join dataframes
#
df_rec['NEWKEY'] = df_rec['NAME'] + df_rec['Year'].astype(str)
df_rush['NEWKEY'] = df_rush['NAME'] + df_rush['Year'].astype(str)
filter = df_rec['NEWKEY'].isin(df_rush['NEWKEY'])

#
# Here's the new dataframe that contains all receivers not already in the rushing dataframe
#
df_rush_rec = df_rec[~filter]

#
# Concatenate the Rushing and the remnant of the Receiving dataframes
#
df3 = pd.concat([df_rush, df_rush_rec], axis=0)
df3.drop('NEWKEY', axis=1,inplace=True) # Get rid of this temporary feature

#
# Next, walk through the new dataframe and for each record, gather following year
# score and previous year score.
#
next_year_list = []
prev_year_list = []

for index,row in df3.iterrows():
    
    name = row['NAME']
    year = row['Year']
    
    next_year_filter = (df3['NAME'] == name) & (df3['Year'].astype(int) == year + 1)
    prev_year_filter = (df3['NAME'] == name) & (df3['Year'].astype(int) == year - 1)
    
    next_year_list.append(df3[next_year_filter]['Score'].values)
    prev_year_list.append(df3[prev_year_filter]['Score'].values)
    
#
# Here we evaluate every record and determine if it is a "high-value" example
# or a "low-value" example.
#
df3['Target'] = 0 # Initialize new feature to zero
target_list = []

for index,row in df3.iterrows():    
    if row['Following Year Value'] >= target_value:
        target_list.append(1)
    else:
        target_list.append(0)
    
#
# Capture the lists we built above into the new dataframe
#
    
df3['Prev Year Value'] = prev_year_list
df3['Following Year Value'] = next_year_list
df3['Prev Year Value'] = df3['Prev Year Value'].str[0].fillna(0)
df3['Following Year Value'] = df3['Following Year Value'].str[0].fillna(0)
df3['Target'] = target_list
    

## Save Data

Colaboratory allows us to save data to the "Download" folder of the computer through this approach.

In [0]:
from google.colab import files

filename_out = 'df3.csv'

df3.to_csv(filename_out)  # Save to .csv format
files.download(filename_out) # Export to Downloads folder